In [1]:
import pandas as pd
import sys
!{sys.executable} -m pip install folium

In [2]:
df = pd.read_csv('../data/interim/restaurants/yelp_sanfrancisco.csv')

In [3]:
df.head()

,Unnamed: 0,ids,names,rating,review_count,price,transactions,is_closed,type_food,type_food_alias,latitude,longitude,country,state,city,zip_code,transactions_
0,0,lJAGnYzku5zSaLnQ_T6_GQ,Brenda's French Soul Food,4.0,11052,$$,['delivery'],False,"Breakfast & Brunch, Southern, Cajun/Creole","breakfast_brunch, southern, cajun",37.782902,-122.419043,US,CA,San Francisco,94102,['delivery']
1,1,NIxLwW9mdYegtqhxmXjoaQ,Sunset Squares Pizza,4.0,9,NaN,[],False,Pizza,pizza,37.776902,-122.415244,US,CA,San Francisco,94103,NaN
2,2,AMhxom9yhQjTkFRMTKk_IA,WingSlut,4.5,6,NaN,[],False,"Chicken Wings, Fast Food, Asian Fusion","chicken_wings, hotdogs, asianfusion",37.776775,-122.416414,US,CA,San Francisco,94103,NaN
3,3,vmZ-bgnM5B76NUpegAjy1g,Hotbird,4.5,129,NaN,[],False,"Sandwiches, Chicken Shop, Burgers","sandwiches, chickenshop, burgers",37.776800,-122.416264,US,CA,San Francisco,94103,NaN
4,4,uEeWn6sPq-giWvfE9uhQ7A,Dottie's True Blue Cafe,4.0,4533,$$,"['pickup', 'delivery']",False,"American (Traditional), Breakfast & Brunch, Cafes","tradamerican, breakfast_brunch, cafes",37.781705,-122.409840,US,CA,San Francisco,94103,"['pickup', 'delivery']"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656 entries, 0 to 1655
Data columns (total 17 columns):
Unnamed: 0         1656 non-null int64
ids                1656 non-null object
names              1656 non-null object
rating             1656 non-null float64
review_count       1656 non-null int64
price              1233 non-null object
transactions       1656 non-null object
is_closed          1656 non-null bool
type_food          1656 non-null object
type_food_alias    1656 non-null object
latitude           1656 non-null float64
longitude          1656 non-null float64
country            1656 non-null object
state              1656 non-null object
city               1656 non-null object
zip_code           1656 non-null int64
transactions_      1352 non-null object
dtypes: bool(1), float64(3), int64(3), object(10)
memory usage: 208.7+ KB


In [5]:
import folium
# San Francisco latitude and longitude values
latitude = 37.77
longitude = -122.42

folium_map = folium.Map(location=[latitude, longitude],
                        zoom_start=13,
                        tiles="OpenStreetMap")
folium_map

In [6]:
# Here I'm using this geo data with the information of the city by zipcode. 
# To replicate this, we need the same geo-data for the city of interest.
sf_neighborhood_geo = 'geo_data/SanFrancisco_Neighborhoods.json'

# Create map
sf_map = folium.Map(
       location=[latitude,longitude],
       zoom_start=12)

# Use json file 
folium.Choropleth(
       geo_data=sf_neighborhood_geo,
       data=df,
       columns=['zip_code', 'review_count'],
       key_on='feature.properties.id',
       fill_color='YlOrRd',
       fill_opacity='0.7',
       line_opacity='0.3').add_to(sf_map)

folium.LayerControl().add_to(sf_map)

# display the map
sf_map

In [7]:
m = folium.Map(location=[latitude, longitude], zoom_start=12)

folium.Choropleth(
       geo_data=sf_neighborhood_geo,
       data=df,
       columns=['zip_code', 'rating'],
       key_on='feature.properties.id',
       fill_color='YlOrRd',
       fill_opacity='0.7',
       line_opacity='0.3').add_to(m)

folium.LayerControl().add_to(m)

m

In [8]:
folium_map = folium.Map(location=[latitude, longitude],
                        zoom_start=13,
                        tiles="OpenStreetMap")

for i in range(0,len(df)):
    marker = folium.CircleMarker(location=[df["latitude"][i],df["longitude"][i]],radius=5, color="blue", fill=True)
    marker.add_to(folium_map)

folium_map

In [9]:
from folium.plugins import HeatMap

max_amount = float(df['review_count'].max())
min_amount = float(df['review_count'].min())

folium_hmap = folium.Map(location=[latitude, longitude],
                        zoom_start=13,
                        tiles="OpenStreetMap")

hm_wide = HeatMap( list(zip(df['latitude'], df['longitude'], df['review_count'])),
                   min_opacity=0.2,
                   max_val=max_amount,
                   min_val=min_amount,
                   radius=8, blur=6, 
                   max_zoom=15, 
                 )

folium_hmap.add_child(hm_wide)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  from ipykernel import kernelapp as app


In [10]:
# TO DO: split by type of food, type of transactions. 
# Identify manually residential zones, recreational zones, offices districts and low and high density regions. 
# Use mobility data from foursquare and google mobility to determine the zones with more and less traffic and 
# movement. Finally, draw markers and geometrical zones using folium, inspect what types of restaurants are in that 
# zone (by type of food, ranking, price (% of expensive, cheap), type of transactions allowed 
# (% using only curbside, only delivery, etc)).
# Display Report with results.